In [402]:
import os
import torch
import torchvision
import numpy
import matplotlib.pyplot as plt
from torchvision import transforms, utils
from fastbook import *
from fastai.vision.widgets import *
from network import U_Net

UNet web APP Mark 1

In [403]:
path = Path()
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()

In [404]:
unet = U_Net(img_ch=3,output_ch=1)
unet.train(False)
unet.eval()

state_dict = torch.load('.\\U_Net-4-16.pth')  
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] # remove `module.`，表面从第7个key值字符取到最后一个字符，正好去掉了module.
    new_state_dict[name] = v #新字典的key值对应的value为一一对应的值。 
# load params
unet.load_state_dict(new_state_dict)

<All keys matched successfully>

In [405]:
Transform = []
Transform.append(transforms.Resize((224,224)))
Transform.append(transforms.ToTensor())
Transform = transforms.Compose(Transform)

In [406]:
# def on_data_change(change):
#     img = PILImage.create(btn_upload.data[-1])
#     out_pl.clear_output()
#     with out_pl: display(img)
#     img = Transform(img)
#     img = img.unsqueeze(0)
#     out = unet(img)
#     out = out.detach().numpy()
#     out = out.squeeze()
#     plt.imshow(out,cmap = plt.cm.gray)
#     with out_pl:  plt.show()
#     torchvision.utils.save_image(out, 'test.png')

In [407]:
def on_data_change(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img)
    img = Transform(img)
    img = img.unsqueeze(0)
    out = unet(img)
    torchvision.utils.save_image(out, 'test.png')
    out = out.squeeze(0)
    out = transforms.ToPILImage()(out)
    with out_pl: display(out)
        

In [408]:
btn_upload.observe(on_data_change, names=['data'])

In [409]:
display(VBox([widgets.Label('上传图片'), btn_upload, out_pl]))